In [123]:
# This is just a visual thing, I don't like how Jupyter usually displays things
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Data processing

In [2]:
import pandas as pd
PARSING_EN_TRAIN = 'DRS_parsing-master/parsing/layer_data/gold/en/dev.conll'
df = pd.DataFrame(columns = ["sentence", "word", "sym", "sem", "sns"])

def read_input(in_file):
    examples = []
    id = ''
    with open("data.txt") as f:
        for line in f.readlines():
            if '#' in line:
                id = line.strip().split('=')[1]
            elif len(line) > 1:
                sp = line.strip().split('\t')
                df.loc[len(df)] = [id, sp[0], sp[1], sp[2], sp[4]]
    return df

dat = read_input(PARSING_EN_TRAIN)
display(dat)

,sentence,word,sym,sem,sns
0,p00/d2862,Tom,tom,PER,male.n.02
1,p00/d2862,was,be,PST,O
2,p00/d2862,carrying,carry,EXG,carry.v.01
3,p00/d2862,a,a,DIS,O
4,p00/d2862,bucket,bucket,CON,bucket.n.01
...,...,...,...,...,...
5418,p10/d2454,.,.,NIL,O
5419,p10/d2335,Tom,tom,PER,male.n.02
5420,p10/d2335,was,be,PST,O
5421,p10/d2335,unarmed,unarmed,IST,unarmed.a.01


# Count occurrences of each word, respective to each word sense
### Make dataframe of all unique words and their total counts (not respective to sense)

In [3]:
unique_words = dat['word'].value_counts().rename_axis('unique_word').reset_index(name='total_count')
display(unique_words)

,unique_word,total_count
0,.,776
1,I,177
2,the,176
3,Tom,173
4,is,168
...,...,...
1557,benefited,1
1558,keyboards,1
1559,roses,1
1560,banana,1


### Per unique word, get all unique senses it's used with and their counts.  Store in another dataframe. Also, store all ambiguous words in another dataframe. 

In [4]:
ambiguous_words = pd.DataFrame(columns = ['word', 'sense', 'count', 'normalize'])
word_df = pd.DataFrame(columns = ['word', 'sense', 'count', 'normalize' ])

# Loop over unique words and get their unique senses
for word in unique_words['unique_word']:
    ambiguous = False
    word_dis = dat[dat['word'] == word]
    sense_counts = word_dis['sns'].value_counts().rename_axis('sense').reset_index(name='counts')
    norm= word_dis['sns'].value_counts(normalize=True).rename_axis('sense').reset_index(name='normalize')
    #display(sense_counts)
    #display(norm)
    new=pd.merge(sense_counts, norm, on=['sense'], how='left')
    #display(norm)
    if len(sense_counts) > 1:
        ambiguous = True
    
    # Store in dataframe
    for i, row in new.iterrows():
        word_df.loc[len(word_df)] = [word, row['sense'], row['counts'], row['normalize']]
        if ambiguous:
            ambiguous_words.loc[len(ambiguous_words)] = [word, row['sense'], row['counts'], row['normalize']]
            
display(word_df)
display(ambiguous_words)

,word,sense,count,normalize
0,.,O,776,1.00000
1,I,O,177,1.00000
2,the,O,176,1.00000
3,Tom,male.n.02,172,0.99422
4,Tom,O,1,0.00578
...,...,...,...,...
1771,benefited,benefit.v.01,1,1.00000
1772,keyboards,keyboard.n.01,1,1.00000
1773,roses,rose.n.01,1,1.00000
1774,banana,banana.n.02,1,1.00000


,word,sense,count,normalize
0,Tom,male.n.02,172,0.994220
1,Tom,O,1,0.005780
2,is,O,125,0.744048
3,is,be.v.02,19,0.113095
4,is,be.v.01,12,0.071429
...,...,...,...,...
366,home,home.n.01,1,0.500000
367,drive,drive.v.01,1,0.500000
368,drive,drive.v.03,1,0.500000
369,make,make.v.16,1,0.500000


In [5]:
# Some examples
display(word_df[word_df['word']=='Tom'])
display(word_df[word_df['word']=='is'])

,word,sense,count,normalize
3,Tom,male.n.02,172,0.99422
4,Tom,O,1,0.00578


,word,sense,count,normalize
5,is,O,125,0.744048
6,is,be.v.02,19,0.113095
7,is,be.v.01,12,0.071429
8,is,be.v.08,4,0.023810
9,is,be.v.03,4,0.023810
10,is,be.v.06,3,0.017857
11,is,be.v.13,1,0.005952


# Naive Bayesian classifier